In [ ]:
using Pkg

In [ ]:
Pkg.add("GraphPlot")
Pkg.add("LightGraphs")
Pkg.add("IterTools")
Pkg.add("Roots")

In [ ]:
using Statistics # needed for the mean function

# WQG

In [ ]:
include("wqg_wr.jl")
include("wqg_wr-temporary.jl")

# parameters

In [ ]:
a = 10.0^(0)
b = 1.0
c = 0.6 # caustic constraint parameter
ϵ = 0.0;

### skeleton

3D lattice graph here

In [ ]:
# set skeleton graph
### z1 direction ordinary, the rest directions periodic
z1, z2, z3 = 5,3,3
lgs = [z1,z2,z3]
D = length(lgs)
d = D-1
sk = LightGraphs.SimpleGraphs.grid(lgs[2:D], periodic=true)
sk = cartesian_product(path_graph(z1),sk)

# set interior and boundary edges
levv_all = l_evv_vertices(sk,vertices(sk))
# interior edges:
bn1 = prod(lgs[2:D]) # no. vertices on one boundary
levv_i = l_evv_vertices(sk,bn1+1:bn1*(z1-1))
# boundary edges:
levv_b = setdiff(levv_all,levv_i);

gplot(sk,nodelabel=1:nv(sk))

### initial data

In [ ]:
# set boundary configurations:
wv = 10.0^(.5) 
rv = 0.2
wfix = value_hom(sk, D, wv)
rfix = value_hom(sk, D, rv)

# set the variable edges:
levv = levv_all
le = length(levv)
ndim=2*le
#ndim = 2

# set fixed volume
### volume is fixed here
vol = 1000.0
# set initial
wi, wf = 0.01, 50
rst = rv
wst = solve_w_volfix(sk,D,b,levv,wfix,rfix,vol,rst,wi,wf)

# starting configuration
pstart = zeros(ndim)
pstart[1:le] .+= wst
pstart[1+le:end] .+= rst
wstart = set_value(wfix, set_value(sk, levv, pstart[1:le])) 
rstart = set_value(rfix, set_value(sk, levv, pstart[1+le:end]))

burnin=2000
nsteps=100000

#scaling
ws = 1
rs = 1

# proposal jump sizes
#jump_w = 1.0/ws
jump_r = 1.0/10;

# dstn
jump_r *= 10.
#jump_r *= 3.

njumps = 2

In [ ]:
log( camp_VG_wr_measure(sk, D, wstart, rstart, ws, rs, a, ϵ, levv_i, levv_b) ) + ( length(levv_i) * log(d) )

### mc run

In [ ]:
chain = zeros(nsteps, ndim)
chain_propose_pars = zeros(nsteps, ndim)
chain_logamp = zeros(nsteps)
chain_propose_logamp = zeros(nsteps)
chain_idx_r = zeros(nsteps, njumps)
chain_idx_w = zeros(nsteps, njumps)
chain_propose_r = zeros(nsteps, njumps)
chain_propose_w = zeros(nsteps, njumps)
chain_old_r = zeros(nsteps, njumps)
chain_old_w = zeros(nsteps, njumps)
chain_gap = zeros(nsteps)

# initial parameters
pars = copy(pstart)
naccepted = 0
wvalue = copy(wstart)
rvalue = copy(rstart)

# initial log-posterior.
# logamp = ln_wqg_wr(sk, levv, pars::Vector{Float64}, wfix, rfix, a, b, levv_b)  
logamp = log( camp_VG_wr_measure(sk, D, wvalue, rvalue, ws, rs, a, ϵ, levv_i, levv_b) ) + ( length(levv_i) * log(d) )

## can make more local:
wpropose = set_value(wvalue, set_value(sk, levv, pars[1:le]))
rpropose = set_value(rvalue, set_value(sk, levv, pars[le+1:2*le]))

nok = 0
for step in 1:nsteps
    if (step % 1000 == 0)
        print(step, ' ')
        flush(stdout)
    end
    pars_new = copy(pars)

    # ratio = amp_propose/amp_old
    rn, ro = 1.0, 1.0
    
    for jump in 1:njumps
        # generate random edges for moves
        idx_r = rand(1:le)
        #idx_w = rand(1:le)
        idx_w = idx_r

        evv_j = levv[idx_r]
        evv_l = levv[idx_w]
        # apply constraints: 0<w and r<c^2*pi^2
        r_p = pars[le+idx_r] + randn() * jump_r
        if (jump == 1)
            w_p = w_new(sk, D, evv_j, evv_l, wvalue, rvalue, r_p)
        else
            w_p = w_new(sk, D, evv_j, evv_l, wpropose, rpropose, r_p)
        end
        if r_p <= c^2 * pi^2 && w_p > 0
            nok += 1
            pars_new[idx_w] = w_p
            pars_new[le+idx_r] = r_p
        end
        chain_propose_r[step, jump] = pars_new[le+idx_r]
        chain_propose_w[step, jump] = pars_new[idx_w]
        chain_old_r[step, jump] = pars[le+idx_r]
        chain_old_r[step, jump] = pars[idx_w]
        chain_idx_w[step, jump] = idx_w
        chain_idx_r[step, jump] = idx_r

        # compute log-posterior at new parameters   
        # edges needing update:
        levv_new = union(l_evv_vertices(sk,evv_l.v1),l_evv_vertices(sk,evv_l.v2),[evv_j])
        # proposed w and r configurations:
        ## can make more local:
        wpropose = set_value(wvalue, set_value(sk, levv, pars_new[1:le]))
        rpropose = set_value(rvalue, set_value(sk, levv, pars_new[le+1:2*le]))
    
        for i in levv_new
            α = corners_α_w(sk, D, i, a, wvalue)
            α_new = corners_α_w(sk, D, i, a, wpropose)
            if i in levv_i
                campEdge = campEdge_wr_measure(sk, D, wvalue, rvalue, ws, rs, i, α, ϵ)
                campEdge_new = campEdge_wr_measure(sk, D, wpropose, rpropose, ws, rs, i, α_new, ϵ)
            else 
                campEdge = campEdge_b(sk, D, wvalue, rvalue, i, α)
                campEdge_new = campEdge_b(sk, D, wpropose, rpropose, i, α_new)
            end
            rn *= campEdge_new
            ro *= campEdge
        end

    end
    chain_propose_pars[step,:] = pars_new::Vector{Float64}

    # amp_new:
    # The rn and ro value are moderate (> 1e-3, < 1e3)
    logamp_new = logamp + log(rn/ro)
    chain_propose_logamp[step] = logamp_new

    gap = rand(Float64)
    chain_gap[step] = gap
    if exp(logamp_new - logamp) >= gap
        logamp = logamp_new
        pars = pars_new
        naccepted += 1
        wvalue = set_value(wstart, set_value(sk, levv, pars[1:le]))
        rvalue = set_value(rstart, set_value(sk, levv, pars[le+1:2*le]))
    end
    chain[step,:] = pars::Vector{Float64}
    chain_logamp[step] = logamp
end
# (s,ρ) configurations
chain_sρ = copy(chain)
for i in 1:nsteps
    for j in 1:le
        chain_sρ[i,j] = 1/chain[i,j]
        chain_sρ[i,j+le] = d*chain[i,j]^2*chain[i,j+le]
    end
end

# data display

In [ ]:
naccepted, naccepted/nsteps, nok/nsteps/njumps

In [ ]:
using FITSIO

In [ ]:
nsteps

In [ ]:
chain_w = zeros(Float32, le, nsteps)
chain_r = zeros(Float32, le, nsteps)
chain_w[:,:] = transpose(chain[:, 1:le])
chain_r[:,:] = transpose(chain[:,le+1:end]);

In [ ]:
f = FITS("chain.fits", "w");
data = Dict("chain_w"=>chain_w, "chain_r"=>chain_r);
write(f, data)
close(f)

In [ ]:
tstvolume(nsteps)

In [ ]:
plot(chain_logamp, label = "logamp")

In [ ]:
histogram(chain_propose_r - chain_old_r)

In [ ]:
accepted = (chain_logamp .= chain_propose_logamp)

In [ ]:
lo =  0.
hi =  3.0
step = 0.25
nbins = Int(1 + (hi - lo) / step)

In [ ]:
diff = chain_propose_r - chain_old_r
diff = hypot.(diff[:,1], diff[:,2])
binvals = lo:step:hi
nstep = zeros(length(binvals))
nacc  = zeros(length(binvals))
for j in 1:length(diff)
d = diff[j]
    if ((d < lo) || (d > hi))
        continue
    end
    bin = Int(trunc((d - lo) / step))
    nstep[bin+1] += 1
    if (chain_propose_logamp[j] == chain_logamp[j])
        nacc[bin+1] += 1
    end
end    

In [ ]:
plot(binvals, nacc./nstep)

In [ ]:
mean(chain_propose_logamp[2:nsteps] - chain_logamp[1:nsteps-1])

In [ ]:
std(chain_propose_logamp[2:nsteps] - chain_logamp[1:nsteps-1])

In [ ]:
ndims(chain)

In [ ]:
size(chain)

In [ ]:
length(Set(chain[:,1]))

In [ ]:
plot(chain[:,1])
plot!(chain[:,2])
plot!(chain[:,3])
plot!(chain[:,4])
plot!(chain[:,100])

In [ ]:
plot(chain[:,127])
plot!(chain[:,128])
plot!(chain[:,129])
plot!(chain[:,130])
plot!(chain[:,226])

In [ ]:
nsteps

In [ ]:
histogram(vec(chain[1000,         le+1:end]), label="r values (step 1000)")
histogram!(vec(chain[Int(nsteps/2),le+1:end]), label="r values (halfway)")
histogram!(vec(chain[nsteps,       le+1:end]), label="r values (final step)")

In [ ]:
histogram(vec(chain[:,1:126]), label="w values")

In [ ]:
Nburn = 1000

In [ ]:
histogram(log.(vec(chain[Nburn:end,1:126])), label="log(w) samples")
#savefig("w-samples-1.png")

In [ ]:
histogram(vec(chain[Nburn:end,127:end]), label="r values")

In [ ]:
histogram(vec(chain[Nburn:end,127:end]), label="r values", bins=range(-3, 5, length=50))
#yaxis!(:log10)

In [ ]:
histogram(vec(chain[Nburn:end,127:4:end]), label="r values", bins=range(-3, 5, length=50))
histogram!(vec(chain[Nburn:end,128:4:end]), label="r values", bins=range(-3, 5, length=50))
histogram!(vec(chain[Nburn:end,129:4:end]), label="r values", bins=range(-3, 5, length=50))
histogram!(vec(chain[Nburn:end,130:4:end]), label="r values", bins=range(-3, 5, length=50))
yaxis!("Number of MCMC samples")
title!("r samples split into 4 sets")
#savefig("r-samples-1.png")

In [ ]:
scatter(chain[90000:end,2], chain[90000:end,128]) #, zcolor=chain_logamp[90000:end])
xaxis!("w", :log10)
yaxis!("r")

In [ ]:
mean(wvalue)

In [ ]:
mean(rvalue)

In [ ]:
var(wvalue)

In [ ]:
var(rvalue)

# Search peaks: amplitudes only go up in mcmc

### initial data

In [ ]:
# set boundary configurations:
wv = 10.0^(.5) 
rv = 0.2
wfix = value_hom(sk, D, wv)
rfix = value_hom(sk, D, rv)

# set the variable edges:
levv = levv_all
le = length(levv)
ndim=2*le
#ndim = 2

# set fixed volume
vol = 1000.0
# set initial
wi, wf = 0.01, 50
rst = rv
wst = solve_w_volfix(sk,D,b,levv,wfix,rfix,vol,rst,wi,wf)

# starting configuration
pstart = zeros(ndim)
pstart[1:le] .+= wst
pstart[1+le:end] .+= rst
wstart = set_value(wfix, set_value(sk, levv, pstart[1:le])) 
rstart = set_value(rfix, set_value(sk, levv, pstart[1+le:end]))

burnin=2000
nsteps=5000

#scaling
ws = 1
rs = 1

# proposal jump sizes
jump_w = 1.0/ws
jump_r = 1.0/10;

In [ ]:
log( camp_VG_wr_measure(sk, D, wstart, rstart, ws, rs, a, ϵ, levv_i, levv_b) ) + ( length(levv_i) * log(d) )

In [ ]:
wstart

In [ ]:
rstart

### mc run: chain only increases amplitude

In [ ]:
chain = []
chain_propose_pars = []
chain_logamp = []
chain_propose_logamp = []
chain_idx_r = []
chain_idx_w = []
chain_gap = []

# initial parameters
pars = copy(pstart)
naccepted = 0
wvalue = copy(wstart)
rvalue = copy(rstart)

# initial log-posterior.
#logamp = ln_wqg_wr(sk, levv, pars::Vector{Float64}, wfix, rfix, a, b, levv_b)  
logamp = log( camp_VG_wr_measure(sk, D, wvalue, rvalue, ws, rs, a, ϵ, levv_i, levv_b) ) + ( length(levv_i) * log(d) )

for i in 1:nsteps
    pars_new = copy(pars)
    # generate random edges for moves
    idx_r = rand(1:le)
    idx_w = rand(1:le)
    evv_j = levv[idx_r]
    evv_l = levv[idx_w]
    # apply constraints: 0<w and r<c^2*pi^2
    r_p = pars[le+idx_r] + randn() * jump_r
    w_p = w_new(sk, D, evv_j, evv_l, wvalue, rvalue, r_p)
    if r_p <= c^2 * pi^2 && w_p > 0
        pars_new[idx_w] = w_p
        pars_new[le+idx_r] = r_p
    end
    append!(chain_propose_pars, pars_new::Vector{Float64})
    append!(chain_idx_w, idx_w)
    append!(chain_idx_r, idx_r)
        
    # compute log-posterior at new parameters   
    # edges needing update:
    levv_new = union(l_evv_vertices(sk,evv_l.v1),l_evv_vertices(sk,evv_l.v2),[evv_j])
    # proposed w and r configurations:
    ## can make more local:
    wpropose = set_value(wvalue, set_value(sk, levv, pars_new[1:le]))
    rpropose = set_value(rvalue, set_value(sk, levv, pars_new[le+1:2*le]))
    # ratio = amp_propose/amp_old
    rn, ro = 1.0, 1.0
    for i in levv_new
        α = corners_α_w(sk, D, i, a, wvalue)
        α_new = corners_α_w(sk, D, i, a, wpropose)
        if i in levv_i
            campEdge = campEdge_wr_measure(sk, D, wvalue, rvalue, ws, rs, i, α, ϵ)
            campEdge_new = campEdge_wr_measure(sk, D, wpropose, rpropose, ws, rs, i, α_new, ϵ)
        else 
            campEdge = campEdge_b(sk, D, wvalue, rvalue, i, α)
            campEdge_new = campEdge_b(sk, D, wpropose, rpropose, i, α_new)
        end
        rn *= campEdge_new
        ro *= campEdge
    end
    # amp_new:
    logamp_new = logamp + log(rn/ro)
    append!(chain_propose_logamp, logamp_new)

    gap = rand(Float64)
    append!(chain_gap, gap)
    if exp(logamp_new - logamp) >= 1 ## set chain only increases amplitude
        #(exp(logamp_new - logamp) >= min(1,2*gap) )
        logamp = logamp_new
        pars = pars_new
        naccepted += 1
        wvalue = set_value(wstart, set_value(sk, levv, pars[1:le]))
        rvalue = set_value(rstart, set_value(sk, levv, pars[le+1:2*le]))
    end
    append!(chain, pars::Vector{Float64})
    append!(chain_logamp, logamp)
end
# append! makes "chain" a 1-d vector; reshape to a matrix
chain = reshape(chain, (ndim,Int64(length(chain)/ndim)))';
chain_propose_pars = reshape(chain_propose_pars, (ndim,Int64(length(chain)/ndim)))';
# (s,ρ) configurations
chain_sρ = copy(chain)
for i in 1:nsteps
    for j in 1:le
        chain_sρ[i,j] = 1/chain[i,j]
        chain_sρ[i,j+le] = d*chain[i,j]^2*chain[i,j+le]
    end
end

### data display

In [ ]:
naccepted

In [ ]:
tstvolume(nsteps)

In [ ]:
plot(chain_logamp,1:1:nsteps,label = "logamp")

In [ ]:
chain_logamp

In [ ]:
mean(wvalue)

In [ ]:
mean(rvalue)

In [ ]:
var(wvalue)

In [ ]:
var(rvalue)